In [ ]:
# generate random list of 500 cities

In [ ]:
# get weather api data

In [ ]:
# Scatter Plots:

In [ ]:
# scatter plot for Temperature (F) vs. Latitude


In [ ]:
# Explanation

In [ ]:
# scatter plot for Humidity (%) vs. Latitude


In [ ]:
# Explanation

In [ ]:
# scatter plot for Cloudiness (%) vs. Latitude


In [ ]:
# Explanation

In [ ]:
# scatter plot for Wind Speed (mph) vs. Latitude


In [ ]:
# Explanation

In [ ]:
# Linear Regressions:

In [ ]:
# Northern Hemisphere - Temperature (F) vs. Latitude
# Southern Hemisphere - Temperature (F) vs. Latitude

In [ ]:
# Analysis

In [ ]:
# Northern Hemisphere - Humidity (%) vs. Latitude
# Southern Hemisphere - Humidity (%) vs. Latitude

In [ ]:
# Analysis

In [ ]:
# Northern Hemisphere - Cloudiness (%) vs. Latitude
# Southern Hemisphere - Cloudiness (%) vs. Latitude

In [ ]:
# Analysis

In [ ]:
# Northern Hemisphere - Wind Speed (mph) vs. Latitude
# Southern Hemisphere - Wind Speed (mph) vs. Latitude

In [ ]:
# Analysis